# Calculating next token probabilities
This notebook provides sample [EDSL](https://docs.expectedparrot.com/) code for using language models to simulate a survey and calculating next token probabilities for models' responses to survey questions.

[EDSL is an open-source libary](https://github.com/expectedparrot/edsl) for simulating surveys, experiments and other research with AI agents and large language models. 
Before running the code below, please ensure that you have [installed the EDSL library](https://docs.expectedparrot.com/en/latest/installation.html) and either [activated remote inference](https://docs.expectedparrot.com/en/latest/remote_inference.html) from your [Coop account](https://docs.expectedparrot.com/en/latest/coop.html) or [stored API keys](https://docs.expectedparrot.com/en/latest/api_keys.html) for the language models that you want to use with EDSL. Please also see our [documentation page](https://docs.expectedparrot.com/) for tips and tutorials on getting started using EDSL. 

## Research question

In [1]:
from IPython.display import HTML
HTML("""<blockquote class="twitter-tweet"><p lang="en" dir="ltr">Aspirational wealth...doing better than you parents...an &quot;Opportunity Economy!&quot; <br>NO!<br>All are late 20th century neoliberal tropes.<br>Americans today seek financial security.<br>Decent jobs and government policy that will pay for the needs of life and old age.<br>Understand that Democrats! <a href="https://t.co/eR3hbx4wbX">pic.twitter.com/eR3hbx4wbX</a></p>&mdash; Dan Alpert (@DanielAlpert) <a href="https://twitter.com/DanielAlpert/status/1833332263733416127?ref_src=twsrc%5Etfw">September 10, 2024</a></blockquote> <script async src="https://platform.twitter.com/widgets.js" charset="utf-8"></script>""")

## Simulating survey responses
In the steps below we demonstrate how to use EDSL to simulate responses to the above question: 

#### *"Which of the following is more important to you: Financial stability / Moving up the income ladder"* 

### Creating questions
We start by selecting a question type and constructing a question in the relevant template.
[EDSL comes with many common question types](https://docs.expectedparrot.com/en/latest/questions.html) that we can choose from based on the desired form of the response:

In [2]:
from edsl import QuestionMultipleChoice

In [3]:
q = QuestionMultipleChoice(
    question_name = "income_pref",
    question_text = "Which of the following is more important to you: ",
    question_options = ["Financial stability", "Moving up the income ladder"]
)

### Designing AI agents
We can design AI agents with relevant `traits` to answer the question:

In [4]:
from edsl import Agent

In [5]:
a = Agent(traits = {"persona": "You are an American answering a poll from Pew."})

### Selecting language models
[EDSL works with many popular models](https://docs.expectedparrot.com/en/latest/language_models.html) that we can use to generate responses:

In [6]:
from edsl import Model

In [7]:
m = Model("gpt-4o", temperature = 1)

### Running a survey
We administer the question by adding the agent and model and then running it.
We can specify the number of times to administer the question:

In [8]:
results = q.by(a).by(m).run(n = 20)

EDSL comes with [built-in methods for analyzing the dataset of `Results`](https://docs.expectedparrot.com/en/latest/results.html) that is generated:

In [9]:
results.select("income_pref").tally().print(format="rich")

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━┓
┃ value               ┃ count ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━┩
│ Financial stability │ 20    │
└─────────────────────┴───────┘

## Calculating token probabilities
In the above example we specified ***n = 20*** to run the question (with the agent and model) 20 times.

We can also get the probabilities from the model by passing ***logprobs = True*** to the `Model`.

To simplify the token probabilities calculation, we can also specify ***use_code = True*** in the `Question` parameters. 
This will cause the question to be presented to the model with coded options: 0 for "Financial stability" and 1 for "Moving up the income ladder":

In [10]:
m = Model("gpt-4o", temperature = 1, logprobs = True)

In [11]:
q = QuestionMultipleChoice(
    question_name = "income_pref", 
    question_text = "Which of the following is more important to you: ", 
    question_options = ["Financial stability", "Moving up the income ladder"], 
    use_code = True
)

In [12]:
new_results = q.by(a).by(m).run(n = 20)

In [13]:
new_results.select("income_pref").tally().print(format = "rich")

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━┓
┃ value               ┃ count ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━┩
│ Financial stability │ 20    │
└─────────────────────┴───────┘

## Inspecting results
The `Results` include information about all the inputs and outputs relating to the question and response. 

To see a list of all the components that can be accessed and analyzed: 

In [14]:
results.columns

['agent.agent_instruction',
 'agent.agent_name',
 'agent.persona',
 'answer.income_pref',
 'comment.income_pref_comment',
 'generated_tokens.income_pref_generated_tokens',
 'iteration.iteration',
 'model.frequency_penalty',
 'model.logprobs',
 'model.max_tokens',
 'model.model',
 'model.presence_penalty',
 'model.temperature',
 'model.top_logprobs',
 'model.top_p',
 'prompt.income_pref_system_prompt',
 'prompt.income_pref_user_prompt',
 'question_options.income_pref_question_options',
 'question_text.income_pref_question_text',
 'question_type.income_pref_question_type',
 'raw_model_response.income_pref_cost',
 'raw_model_response.income_pref_one_usd_buys',
 'raw_model_response.income_pref_raw_model_response']

We can inspect the `raw_model_response.income_pref_raw_model_response` component to identify next token probabilities:

In [15]:
example = new_results.select("raw_model_response.income_pref_raw_model_response").to_list()[0]

In [16]:
next_token_probs = example['choices'][0]['logprobs']['content'][0]['top_logprobs']
next_token_probs

[{'token': '0', 'bytes': [48], 'logprob': -0.00028982185},
 {'token': '1', 'bytes': [49], 'logprob': -8.25029},
 {'token': ' ', 'bytes': [32], 'logprob': -11.50029}]

### Translating the information

In [18]:
token_info

{'token': ' ', 'bytes': [32], 'logprob': -11.50029}

In [19]:
import math

# Specifying the codes for the answer options and non-responses:
options = {'0': "Financial stability", '1':"Moving up the income ladder", '\n': "Skipped", " ": "Skipped"}

for token_info in next_token_probs:
    option = options[token_info['token']]
    p = math.exp(token_info['logprob'])
    
    print(f"Probability of selecting '{option}' was {p:.3f}")

Probability of selecting 'Financial stability' was 1.000
Probability of selecting 'Moving up the income ladder' was 0.000
Probability of selecting 'Skipped' was 0.000


## Comparing models
We can rerun the survey with other available models.

To see a list of all available models:

In [18]:
# Model.available()

In [20]:
models = [Model(model_name) for model_name, _, _ in Model.available()]

In [21]:
len(models)

151

We know some models will not be appropriate; we can add `print_exceptions = False` to skip the error report:

In [21]:
results_with_many_models = q.by(a).by(models).run(print_exceptions = False)

### Performance
We can check which models did/not answer the question, and filter out the non-responses:

In [22]:
(
    results_with_many_models
    .filter('income_pref is not None')
    .select('income_pref')
    .tally()
    .print(format = "rich")
)

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━┓
┃ value                       ┃ count ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━┩
│ Financial stability         │ 86    │
├─────────────────────────────┼───────┤
│ Moving up the income ladder │ 8     │
└─────────────────────────────┴───────┘

In [23]:
results_with_many_models.filter("income_pref is not None").select("model").print()

model.model
01-ai/Yi-34B-Chat
Austism/chronos-hermes-13b-v2
Gryphe/MythoMax-L2-13b
Gryphe/MythoMax-L2-13b-turbo
HuggingFaceH4/zephyr-orpo-141b-A35b-v0.1
Phind/Phind-CodeLlama-34B-v2
Qwen/Qwen2-72B-Instruct
Qwen/Qwen2-7B-Instruct
Qwen/Qwen2.5-72B-Instruct
Sao10K/L3-70B-Euryale-v2.1


## Posting to the Coop
The [Coop](https://www.expectedparrot.com/explore) is a platform for creating, storing and sharing LLM-based research.
It is fully integrated with EDSL and accessible from your workspace or Coop account page.
Learn more about [creating an account](https://www.expectedparrot.com/login) and [using the Coop](https://docs.expectedparrot.com/en/latest/coop.html).

Here we demonstrate how to post this notebook:

In [24]:
from edsl import Notebook

In [25]:
n = Notebook(path = "next_token_probs.ipynb")

In [26]:
n.push(description = "Example code for calculating next token probabilities", visibility = "public")

{'description': 'Example code for calculating next token probabilities',
 'object_type': 'notebook',
 'url': 'https://www.expectedparrot.com/content/8be8de45-006c-484a-b677-8e3bb25f8ff7',
 'uuid': '8be8de45-006c-484a-b677-8e3bb25f8ff7',
 'version': '0.1.33.dev1',
 'visibility': 'public'}

To update an object at the Coop:

In [29]:
n = Notebook(path = "next_token_probs.ipynb") # resave

In [30]:
n.patch(uuid = "8be8de45-006c-484a-b677-8e3bb25f8ff7", value = n)

{'status': 'success'}